In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pdb as pdb
import time
from IPython.core.debugger import set_trace
import tensorflow as tf

In [2]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [4]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const_2:0", shape=(), dtype=float32)


In [5]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [6]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [7]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

In [8]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In [9]:
# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))


W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


In [10]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
# Note that input_fn will be called while constructing the TensorFlow graph, not while running the graph. 
# What it is returning is a representation of the input data as the fundamental unit of TensorFlow computations, a Tensor (or SparseTensor).
# num_epochs is max value, actual steps are defined in estimator.train()
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=2, num_epochs=150, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=100, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=200, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set. steps are number of batch calls (not epochs)
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_master': '', '_num_ps_replicas': 0, '_model_dir': '/tmp/tmpn4fh09g4', '_is_chief': True, '_save_checkpoints_steps': None, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd62af43278>, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_session_config': None, '_service': None, '_task_id': 0, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpn4fh09g4/model.ckpt.
INFO:tensorflow:loss = 10.0, step = 1
INFO:tensorflow:global_step/sec: 515.067
INFO:tensorflow:loss = 0.24503, step = 101 (0.203 sec)
INFO:tensorflow:global_step/sec: 739.701
INFO:tensorflow:loss = 0.0134421, step = 201 (0.136 sec)
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmpn4fh09g4/model

In [11]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_master': '', '_num_ps_replicas': 0, '_model_dir': '/tmp/tmp9zg3y4iy', '_is_chief': True, '_save_checkpoints_steps': None, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5fc303cc0>, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_session_config': None, '_service': None, '_task_id': 0, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp9zg3y4iy/model.ckpt.
INFO:tensorflow:loss = 56.2281217708, step = 1
INFO:tensorflow:global_step/sec: 614.682
INFO:tensorflow:loss = 0.110450641029, step = 101 (0.170 sec)
INFO:tensorflow:global_step/sec: 484.68
INFO:tensorflow:loss = 0.00939208302401, step = 201 (0.205 sec)
INFO:tensorflow:global_step/sec: 583.056
INFO:te

In [12]:
# Mnist beginners tutorial
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print("mnist {}".format(mnist))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
mnist Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd5fc6edeb8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd5fc6edef0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fd5fc6edf28>)


In [13]:
# placeholder for input / output variables first dim "None" for undefined number of samples, second flattened vectors
x = tf.placeholder(tf.float32, [None, 784])
# variables for weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# define model

y = tf.nn.softmax(tf.matmul(x, W) + b)

# define placeholder for one-hot encoded labels, first dim "None" for undefined number of samples
y_ = tf.placeholder(tf.float32, [None, 10])
# define error function (theoretically - but numerically unstable)
cross_entropy_old = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# define error function by shortcutting softmax plus cross entropy directly on unnormalised linear data
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# define training step, using gradient descent and the cross-entropy loss function defined above 
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# defining session - as default session (so "sess" must not called explicity subsequently)
sess = tf.InteractiveSession()
# actually initialise all gobal variables as set before in graphs
tf.global_variables_initializer().run()
# train
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
# check for correctness of prediction, compare model output y
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
print("y_ {}".format(y_[0]))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

y_ Tensor("strided_slice:0", shape=(10,), dtype=float32)
0.9066


In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
FLAGS = None

In [15]:
# initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients
# The generated values follow a normal distribution with specified mean and standard deviation, 
# except that values whose magnitude is more than 2 standard deviations from the mean are dropped and re-picked.
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# define constant bias
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# Computes a 2-D convolution given 4-D input and filter tensors. returning 4D tensor of same type as x
# ? means zero padding überall 2, oder ?
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# first convolutional layer with 32 nodes --> woher die 32?? 
# initialise weights and bias (slightly randomized)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# define tensor representing input layer: reshape flattened 748 Vector to 4d tensor 28 x 28 image matrixes
# and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve, same size (padding, stride...), reduce size to 14x14 by maxpooling
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# define second layer concolving same size, max-pooling --> 7x7   64 features ??
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# convolve, same size (padding, stride...), reduce size to 14x14 by maxpooling
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# we add a fully-connected layer with 1024 neurons to allow processing on the entire image.
# We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, 
# and apply a ReLU.
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# implement dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# final fully connected layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(100):
    batch = mnist.train.next_batch(50)
    if i % 50 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print("test")
  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
test
accu done
accu 0.11190000176429749
test accuracy 0.1119
test
accu done
accu 0.09059999883174896
test accuracy 0.0906
test
accu done
accu 0.0892999991774559
test accuracy 0.0893
test
accu done
accu 0.08659999817609787
test accuracy 0.0866
test
